# Dependecies


In [1]:
import pandas as pd 
import numpy as np
import numbers, datetime, warnings, os, re, math
from natsort import natsort_keygen

# Dashboard

* read vcf
* find stack vcf row
* split stack vcf row
* filter vcf by `criteria`
* compare vcf row by row
* label caller type
* save file (4.1, 4.2)


## Read VCF

1. find the starting of the vcf table
2. store the vcf header seperately
3. read vcf as dataframe


In [2]:
def read_vcf(file_path: str) -> tuple[dict, pd.DataFrame]:
    header_dict = {}
    with open(file_path, 'r') as f:
        while True:
            line = f.readline().rstrip()
            if line.startswith('##'):
                key = re.match(r"##(\w+)", line)
                if key:
                    category = key.group(1) 
                    if category not in header_dict:
                        header_dict[category] = [] 
                    header_dict[category].append(line)
                    
            else:
                columns = line.rstrip().split('\t')
                break

        try:
            vcf = pd.read_csv(f, sep="\t", header=None, comment="#", names=columns)
        except pd.errors.EmptyDataError:
            vcf = pd.DataFrame(columns=columns) 
        vcf.columns = columns

    return (header_dict, vcf)

In [3]:
header_dict, vcf = read_vcf('test.filtered.vcf')
header_dict, vcf = read_vcf('test.somatic.indel.Somatic.hc.vcf')

In [4]:
for category, lines in header_dict.items():
    print(f"### {category.upper()} HEADERS ###")
    for line in lines:
        print(line)
    print("\n")

### FILEFORMAT HEADERS ###
##fileformat=VCFv4.1


### SOURCE HEADERS ###
##source=VarScan2


### INFO HEADERS ###
##INFO=<ID=DP,Number=1,Type=Integer,Description="Total depth of quality bases">
##INFO=<ID=SOMATIC,Number=0,Type=Flag,Description="Indicates if record is a somatic mutation">
##INFO=<ID=SS,Number=1,Type=String,Description="Somatic status of variant (0=Reference,1=Germline,2=Somatic,3=LOH, or 5=Unknown)">
##INFO=<ID=SSC,Number=1,Type=String,Description="Somatic score in Phred scale (0-255) derived from somatic p-value">
##INFO=<ID=GPV,Number=1,Type=Float,Description="Fisher's Exact Test P-value of tumor+normal versus no variant for Germline calls">
##INFO=<ID=SPV,Number=1,Type=Float,Description="Fisher's Exact Test P-value of tumor versus normal for Somatic/LOH calls">


### FILTER HEADERS ###
##FILTER=<ID=str10,Description="Less than 10% or more than 90% of variant supporting reads on one strand">
##FILTER=<ID=indelError,Description="Likely artifact due to indel reads at th

In [5]:
header_dict['FORMAT']

['##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
 '##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">',
 '##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read Depth">',
 '##FORMAT=<ID=RD,Number=1,Type=Integer,Description="Depth of reference-supporting bases (reads1)">',
 '##FORMAT=<ID=AD,Number=1,Type=Integer,Description="Depth of variant-supporting bases (reads2)">',
 '##FORMAT=<ID=FREQ,Number=1,Type=String,Description="Variant allele frequency">',
 '##FORMAT=<ID=DP4,Number=1,Type=String,Description="Strand read counts: ref/fwd, ref/rev, var/fwd, var/rev">']

In [6]:
vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,189392,.,ACC,A,.,PASS,DP=206;SOMATIC;SS=2;SSC=79;GPV=1E0;SPV=1.081E-8,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:78:51:0:0%:51,0,27,0","0/1:.:128:82:46:35.94%:82,0,46,0"
1,chr1,939570,.,T,TCCCTGGAGGACC,.,str10,DP=32;SOMATIC;SS=2;SSC=19;GPV=1E0;SPV=1.0195E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:11:8:0:0%:0,8,1,2","0/1:.:21:10:11:52.38%:2,8,0,11"
2,chr1,1013466,.,T,TA,.,PASS,DP=437;SOMATIC;SS=2;SSC=24;GPV=1E0;SPV=3.7671E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:50:2:0:0%:2,0,38,6","1/1:.:387:21:344:94.25%:20,1,300,44"
3,chr1,1035169,.,TG,T,.,PASS,DP=116;SOMATIC;SS=2;SSC=44;GPV=1E0;SPV=3.8026E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:29:12:0:0%:12,0,13,0","0/1:.:87:23:39:62.9%:23,0,39,0"
4,chr1,1043223,.,CCT,C,.,PASS,DP=138;SOMATIC;SS=2;SSC=38;GPV=1E0;SPV=1.4286E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:45:4:0:0%:4,0,41,0","1/1:.:93:8:85:91.4%:8,0,84,1"
...,...,...,...,...,...,...,...,...,...,...,...
7652,chrUn_JTFH01001145v1_decoy,665,.,GC,G,.,str10,DP=246;SOMATIC;SS=2;SSC=19;GPV=1E0;SPV=1.1611E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:116:113:0:0%:60,53,3,0","0/1:.:130:123:7:5.38%:89,34,7,0"
7653,chrUn_JTFH01001219v1_decoy,837,.,C,CCTTCGCCAT,.,PASS,DP=210;SOMATIC;SS=2;SSC=17;GPV=1E0;SPV=1.615E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:76:75:0:0%:30,45,1,0","0/1:.:134:124:9:6.77%:56,68,3,6"
7654,chrUn_JTFH01001223v1_decoy,695,.,C,CGGGGT,.,PASS,DP=43;SOMATIC;SS=2;SSC=23;GPV=1E0;SPV=4.1629E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:20:14:0:0%:13,1,4,1","0/1:.:23:12:9:42.86%:12,0,9,0"
7655,chrUn_JTFH01001243v1_decoy,888,.,GGC,G,.,PASS,DP=33;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.238E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:14:8:0:0%:5,3,1,4","0/1:.:19:8:10:55.56%:1,7,3,7"


In [7]:
def get_number(header_list: list[str]) -> dict[str:str]:
    tmp_dict = {}
    if not isinstance(header_list, list):
        raise TypeError("header_list should be a list of strings")
    
    for v in header_list:
        if not isinstance(v, str):
            continue
        
        match = re.match(r'^##(INFO|FORMAT|FILTER)=<ID=([^,]+),Number=([^,>]+)', v)
        if match:
            category, id_, number = match.groups()
            tmp_dict[id_] = number
    
    return tmp_dict

info_dict = get_number(header_dict['INFO'])
format_dict = get_number(header_dict['FORMAT'])



## Find stack vcf 

In [73]:
def is_stack(row: pd.Series) -> bool:
    return len(row['ALT'].split(',')) >1

def stack_count(row: pd.Series) -> int:
    return len(row['ALT'].split(','))

In [74]:
vcf[vcf.apply(is_stack, axis=1)]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR


## split stack row
1. get the count of stack mutation
1. create `<count>` duplicate row
2. split filter
3. split INFO
    1. split ;
    2. split = 
    3. split ,
4. split NA0001, NA0002...
1. remove stacked row
1. add new row

In [ ]:
def unstack_vcf(vcf: pd.DataFrame) -> pd.DataFrame:
    splitted_rows = []
    stacked_indices = []
    stacked_vcf = vcf[vcf.apply(is_stack, axis=1)]

    for idx, row in stacked_vcf.iterrows():
        stacked_indices.append(idx)
        splitted_rows.extend(split_variant(row))

    vcf = vcf.drop(stacked_indices).reset_index(drop=True)

    if splitted_rows:
        new_vcf = pd.DataFrame(splitted_rows)
        vcf = pd.concat([vcf, new_vcf], ignore_index=True)

    return sorting(vcf)

def sorting(vcf: pd.DataFrame) -> pd.DataFrame:
    return vcf.iloc[vcf.apply(natsort_keygen(key=lambda x: (x['#CHROM'], x['POS'])), axis=1).argsort()].reset_index(drop=True)

def split_variant(row: pd.Series) -> list[pd.Series]: 
    count = stack_count(row)

    splitted_rows = [row.copy() for _ in range(count)]
    alt_alleles = row['ALT'].split(',')
    infos = parse_info(row['INFO'], count)
    
    format = row['FORMAT']
    samples = [parse_sample(sample, format, count) for sample in row.iloc[9:].tolist()]
    
    for i in range(count):
        splitted_rows[i]['ALT'] = alt_alleles[i]
        splitted_rows[i]['INFO'] = infos[i]

        for j, sample_name in enumerate(row.index[9:]):  
            splitted_rows[i][sample_name] = samples[j][i]

    return splitted_rows
    

def parse_info(info: str, count: int) -> list[str]:
    info_list = []
    
    infos = info.split(';')
    for inf in infos:
        kv_pair = re.match(r'(\w+)=([\w\.\-\,\+]+)', inf)
        
        if kv_pair:
            key, value = kv_pair.groups()
            splitted_v: list = value.split(',')
            if info_dict[key] == 'R': # NUMBER = R
                if len(splitted_v) != count+1:
                    raise ValueError(f"Error: {key} should have {count+1} values, got {len(splitted_v)}")
                
                ref_info = splitted_v[0]
                splitted_v.pop(0)
                info_list.append([f'{key}={ref_info},{v_}' for v_ in splitted_v])

                
            elif info_dict[key] == 'A': # NUMENR = A
                if len(splitted_v) != count:
                    raise ValueError(f"Error: {key} should have {count} values, got {len(splitted_v)}")
                
                info_list.append([f'{key}={v_}' for v_ in splitted_v])

                    
            elif info_dict[key] == '.':
                pass

            else:
                expected_count = int(info_dict[key])
                if len(splitted_v) != expected_count:
                    raise ValueError(f"Error: {key} should have {expected_count} values, got {len(splitted_v)}")
                info_list.append([f"{key}={value}" for _ in range(count)])
        else:
            if(int(info_dict[inf]) != 0):
                raise ValueError(f"{inf} is not a FLAG, it has {info_dict[inf]} number of value")
            
            info_list.append([inf for _ in range(count)])


    transposed_info = list(map(list, zip(*info_list)))
    return [';'.join(inf_list) for inf_list in transposed_info]

def parse_sample(sample: str, format: str, count: int) -> list[str]:
    info_list = []
    formats: list[str] = format.split(':')
    sample_infos: list[str] = sample.split(':')

    # check number of formats and sample_infos must match 
    if len(formats) != len(sample_infos):
        raise ValueError(...)

    for fmt, infos in zip(formats, sample_infos):
        splitted_info = infos.split(',')

        if fmt == 'GT' or fmt == 'PGT':
            if infos == '0/0' or infos == '0' or '0|0':
                info_list.append([infos for _ in range(count)])
            else:
                info_list.append(['0/1' for _ in range(count)])
            continue

        if format_dict[fmt] == 'R':
            if len(splitted_info) != count+1:
                raise ValueError(f"Error: {fmt} should have {count+1} values, got {len(splitted_info)}")

            ref_info = splitted_info[0]
            splitted_info.pop(0)
            info_list.append([f'{ref_info},{v_}' for v_ in splitted_info])

        elif format_dict[fmt] == 'A':
            if len(splitted_info) != count:
                raise ValueError(f"Error: {fmt} should have {count} values, got {len(splitted_info)}")
            
            info_list.append(splitted_info)

        else:
            if format_dict[fmt] != 'G' and len(splitted_info) != int(format_dict[fmt]):
                raise ValueError(f"Error: {fmt} should have {int(format_dict[fmt])} values, got {len(splitted_info)}")
            info_list.append([infos for _ in range(count)])
        
    transposed_info = list(map(list, zip(*info_list)))
    return [':'.join(inf_list) for inf_list in transposed_info]

In [76]:
vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,189392,.,ACC,A,.,PASS,DP=206;SOMATIC;SS=2;SSC=79;GPV=1E0;SPV=1.081E-8,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:78:51:0:0%:51,0,27,0","0/1:.:128:82:46:35.94%:82,0,46,0"
1,chr1,939570,.,T,TCCCTGGAGGACC,.,str10,DP=32;SOMATIC;SS=2;SSC=19;GPV=1E0;SPV=1.0195E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:11:8:0:0%:0,8,1,2","0/1:.:21:10:11:52.38%:2,8,0,11"
2,chr1,1013466,.,T,TA,.,PASS,DP=437;SOMATIC;SS=2;SSC=24;GPV=1E0;SPV=3.7671E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:50:2:0:0%:2,0,38,6","1/1:.:387:21:344:94.25%:20,1,300,44"
3,chr1,1035169,.,TG,T,.,PASS,DP=116;SOMATIC;SS=2;SSC=44;GPV=1E0;SPV=3.8026E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:29:12:0:0%:12,0,13,0","0/1:.:87:23:39:62.9%:23,0,39,0"
4,chr1,1043223,.,CCT,C,.,PASS,DP=138;SOMATIC;SS=2;SSC=38;GPV=1E0;SPV=1.4286E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:45:4:0:0%:4,0,41,0","1/1:.:93:8:85:91.4%:8,0,84,1"
...,...,...,...,...,...,...,...,...,...,...,...
7652,chrUn_JTFH01001145v1_decoy,665,.,GC,G,.,str10,DP=246;SOMATIC;SS=2;SSC=19;GPV=1E0;SPV=1.1611E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:116:113:0:0%:60,53,3,0","0/1:.:130:123:7:5.38%:89,34,7,0"
7653,chrUn_JTFH01001219v1_decoy,837,.,C,CCTTCGCCAT,.,PASS,DP=210;SOMATIC;SS=2;SSC=17;GPV=1E0;SPV=1.615E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:76:75:0:0%:30,45,1,0","0/1:.:134:124:9:6.77%:56,68,3,6"
7654,chrUn_JTFH01001223v1_decoy,695,.,C,CGGGGT,.,PASS,DP=43;SOMATIC;SS=2;SSC=23;GPV=1E0;SPV=4.1629E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:20:14:0:0%:13,1,4,1","0/1:.:23:12:9:42.86%:12,0,9,0"
7655,chrUn_JTFH01001243v1_decoy,888,.,GGC,G,.,PASS,DP=33;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.238E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:14:8:0:0%:5,3,1,4","0/1:.:19:8:10:55.56%:1,7,3,7"


In [77]:
vcf_prime = unstack_vcf(vcf)

## FIlter VCF
1. filter == "PASS"
2. get DP, RD, AD, VAF 

In [78]:
header_dict['FORMAT']

['##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
 '##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">',
 '##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read Depth">',
 '##FORMAT=<ID=RD,Number=1,Type=Integer,Description="Depth of reference-supporting bases (reads1)">',
 '##FORMAT=<ID=AD,Number=1,Type=Integer,Description="Depth of variant-supporting bases (reads2)">',
 '##FORMAT=<ID=FREQ,Number=1,Type=String,Description="Variant allele frequency">',
 '##FORMAT=<ID=DP4,Number=1,Type=String,Description="Strand read counts: ref/fwd, ref/rev, var/fwd, var/rev">']

In [79]:
def filter_PASS(vcf: pd.DataFrame) -> pd.DataFrame:
    vcf = vcf[vcf.FILTER == 'PASS']
    return vcf.reset_index(drop=True)

In [123]:
def get_sample_info(vcf: pd.Series) -> tuple[int, int, float]:
    keys= vcf["FORMAT"].split(":")
    values = vcf.iloc[-1].split(":")
    sample_info = {k:v for k,v in zip(keys,values)}

    DP = int(sample_info["DP"]) if "DP" in sample_info else 0

    if "AD" in sample_info:
        AD_values = list(map(int, sample_info["AD"].split(",")))
        AD = AD_values[1] if len(AD_values) > 1 else AD_values[0]
    elif "AO" in sample_info:
        AD = int(sample_info["AO"])
    else:
        AD = 0

    if "AF" in sample_info:
        try:
            VAF = float(sample_info["AF"])
        except ValueError:
            VAF = AD / DP if DP > 0 else 0.0
    else:
        VAF = AD / DP if DP > 0 else 0.0

    return AD, DP, VAF

def get_normal_info(vcf: pd.Series) -> tuple[int, int, float]:
    keys= vcf["FORMAT"].split(":")
    values = vcf.iloc[9].split(":")
    sample_info = {k:v for k,v in zip(keys,values)}

    DP = int(sample_info["DP"]) if "DP" in sample_info else 0

    if "AD" in sample_info:
        AD_values = list(map(int, sample_info["AD"].split(",")))
        AD = AD_values[1] if len(AD_values) > 1 else AD_values[0]
    elif "AO" in sample_info:
        AD = int(sample_info["AO"])
    else:
        AD = 0

    if "AF" in sample_info:
        try:
            VAF = float(sample_info["AF"])
        except ValueError:
            VAF = AD / DP if DP > 0 else 0.0
    else:
        VAF = AD / DP if DP > 0 else 0.0

    return AD, DP, VAF

def filter_VAF(vcf: pd.Series, AD_thres:int = 3,DP_thres: int = 30, VAF_thres: float = 0.05, germline = False ) -> bool:
    AD, DP, VAF = get_sample_info(vcf) if not germline else get_normal_info(vcf)
    if AD < AD_thres:
        return False
    if DP < DP_thres:
        return False
    if VAF < VAF_thres:
        return False
    return True


## Merge!!!
1. Chrom, Pos, Ref, Alt
1. 

In [124]:
def merge_somatic(mutect:pd.DataFrame, varscan: pd.DataFrame) -> pd.DataFrame:
    varscan_l = varscan[['#CHROM', 'POS', 'REF', 'ALT']]
    return pd.merge(mutect, varscan_l, on=['#CHROM', 'POS', 'REF', 'ALT'], how='inner')

def merge_germline(haplotypecaller: pd.DataFrame, varscan: pd.DataFrame) -> pd.DataFrame:
    varscan_l = varscan[['#CHROM', 'POS', 'REF', 'ALT']]
    return pd.merge(haplotypecaller, varscan_l, on=['#CHROM', 'POS', 'REF', 'ALT'], how='inner')

## Compiling

In [146]:
def build_vcf(path: str) -> pd.DataFrame:
    header_dict, vcf = read_vcf(path)
    info_dict = get_number(header_dict['INFO']) # for global usage
    format_dict = get_number(header_dict['FORMAT']) # for global usage
    vcf = unstack_vcf(vcf)
    vcf = filter_PASS(vcf).reset_index(drop=True)
    vcf = vcf[vcf.apply(filter_VAF, axis=1)].reset_index(drop=True)
    return vcf

def build_gvcf(path: str, PASS: bool=True, VAF_thres: float = 0.1) -> pd.DataFrame:
    header_dict, vcf = read_vcf(path)
    info_dict = get_number(header_dict['INFO']) # for global usage
    format_dict = get_number(header_dict['FORMAT']) # for global usage
    vcf = unstack_vcf(vcf)
    if PASS:
        vcf = filter_PASS(vcf).reset_index(drop=True)
    vcf = vcf[vcf.apply(filter_VAF, axis=1, VAF_thres=VAF_thres, germline = True)].reset_index(drop=True)
    return vcf

def read_mutect(path: str) -> pd.DataFrame:
    return build_vcf(path)

def read_varscan(snp_path: str, indel_path) -> pd.DataFrame: 
    snp_vcf = build_vcf(snp_path)
    indel_vcf = build_vcf(indel_path)
    return sorting(pd.concat([snp_vcf,indel_vcf]))

def read_varscan_g(snp_path: str, indel_path) -> pd.DataFrame: 
    snp_vcf = build_gvcf(snp_path)
    indel_vcf = build_gvcf(indel_path)
    return sorting(pd.concat([snp_vcf,indel_vcf]))

def read_haplotypecaller(path: str) -> pd.DataFrame:
    return build_gvcf(path, PASS=False)

## test

### Somatic 

In [ ]:
header_dict['QUAL']

{'fileformat': ['##fileformat=VCFv4.2'],
 'ALT': ['##ALT=<ID=NON_REF,Description="Represents any possible alternative allele at this location">'],
 'FILTER': ['##FILTER=<ID=LowQual,Description="Low quality">'],
 'FORMAT': ['##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">',
  '##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">',
  '##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">',
  '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
  '##FORMAT=<ID=MIN_DP,Number=1,Type=Integer,Description="Minimum DP observed within the GVCF block">',
  '##FORMAT=<ID=PGT,Number=1,Type=String,Description="Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another">',
  '##FORMAT=<ID=PID,Number=1,Type=String,Description="Physical phasing ID information, where each unique ID 

In [148]:
header_dict, vcf = read_vcf('test.filtered.vcf')

info_dict = get_number(header_dict['INFO'])
format_dict = get_number(header_dict['FORMAT'])
mutect_vcf = unstack_vcf(vcf)
mutect_vcf = filter_PASS(mutect_vcf).reset_index(drop=True)
mutect_vcf = mutect_vcf[mutect_vcf.apply(filter_VAF, axis=1)].reset_index(drop=True)

mutect_vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,35850153,.,G,T,.,PASS,"CONTQ=41;DP=159;ECNT=1;GERMQ=273;MBQ=38,30;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:61,1:0.030:62:39,0:22,1","0/1:80,5:0.067:85:43,0:37,5"
4,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
...,...,...,...,...,...,...,...,...,...,...,...
173,chrX,76428580,.,CTGAGGGCCTGAGCACCTCCGTGCAGGCCACTCCTGA,C,.,PASS,"CONTQ=93;DP=480;ECNT=1;GERMQ=514;MBQ=35,36;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:174,0:5.561e-03:174:74,0:100,0","0/1:219,49:0.188:268:127,27:92,22"
174,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
175,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"
176,chrX,85003924,.,C,T,.,PASS,"CONTQ=93;DP=483;ECNT=1;GERMQ=753;MBQ=37,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:113,0:8.669e-03:113:58,0:55,0","0/1:185,177:0.489:362:74,75:111,102"


In [140]:
header_dict, vcf = read_vcf('test.somatic.snp.Somatic.hc.vcf')

info_dict = get_number(header_dict['INFO'])
format_dict = get_number(header_dict['FORMAT'])
varscan_snp_vcf = unstack_vcf(vcf)
varscan_snp_vcf = filter_PASS(varscan_snp_vcf).reset_index(drop=True)
varscan_snp_vcf = varscan_snp_vcf[varscan_snp_vcf.apply(filter_VAF, axis=1)].reset_index(drop=True)

varscan_snp_vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,1197765,.,C,G,.,PASS,DP=90;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.4307E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:20:20:0:0%:11,9,0,0","0/1:.:70:53:17:24.29%:23,30,5,12"
1,chr1,1533904,.,C,T,.,PASS,DP=113;SOMATIC;SS=2;SSC=64;GPV=1E0;SPV=3.6675E-7,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:30:30:0:0%:30,0,0,0","0/1:.:83:45:38:45.78%:45,0,38,0"
2,chr1,1966436,.,C,T,.,PASS,DP=80;SOMATIC;SS=2;SSC=32;GPV=1E0;SPV=5.9775E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:24:24:0:0%:14,10,0,0","0/1:.:56:38:18:32.14%:28,10,9,9"
3,chr1,12941584,.,G,A,.,PASS,DP=124;SOMATIC;SS=2;SSC=18;GPV=1E0;SPV=1.4505E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:45:45:0:0%:1,44,0,0","0/1:.:79:70:9:11.39%:0,70,0,9"
4,chr1,16567686,.,A,G,.,PASS,DP=79;SOMATIC;SS=2;SSC=12;GPV=1E0;SPV=5.4743E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:40:40:0:0%:22,18,0,0","0/1:.:39:35:4:10.26%:20,15,1,3"
...,...,...,...,...,...,...,...,...,...,...,...
1102,chrX,150714570,.,C,T,.,PASS,DP=190;SOMATIC;SS=2;SSC=91;GPV=1E0;SPV=6.9417E-10,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:39:39:0:0%:2,37,0,0","0/1:.:151:80:71:47.02%:8,72,4,67"
1103,chrX,152736878,.,T,C,.,PASS,DP=542;SOMATIC;SS=2;SSC=32;GPV=1E0;SPV=6.0385E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:171:161:7:4.17%:68,93,2,5","0/1:.:371:321:49:13.24%:162,159,37,12"
1104,chrX,152736881,.,G,A,.,PASS,DP=541;SOMATIC;SS=2;SSC=36;GPV=1E0;SPV=2.3013E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:169:160:8:4.76%:69,91,2,6","0/1:.:372:315:56:15.09%:159,156,41,15"
1105,chrX,156022024,.,C,T,.,PASS,DP=204;SOMATIC;SS=2;SSC=35;GPV=1E0;SPV=2.8015E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:73:70:3:4.11%:56,14,2,1","0/1:.:131:101:29:22.31%:76,25,15,14"


In [137]:
header_dict, vcf = read_vcf('test.somatic.indel.Somatic.hc.vcf')

info_dict = get_number(header_dict['INFO'])
format_dict = get_number(header_dict['FORMAT'])
varscan_indel_vcf = unstack_vcf(vcf)
varscan_indel_vcf = filter_PASS(varscan_indel_vcf).reset_index(drop=True)
varscan_indel_vcf = varscan_indel_vcf[varscan_indel_vcf.apply(filter_VAF, axis=1)].reset_index(drop=True)


varscan_indel_vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,189392,.,ACC,A,.,PASS,DP=206;SOMATIC;SS=2;SSC=79;GPV=1E0;SPV=1.081E-8,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:78:51:0:0%:51,0,27,0","0/1:.:128:82:46:35.94%:82,0,46,0"
1,chr1,1013466,.,T,TA,.,PASS,DP=437;SOMATIC;SS=2;SSC=24;GPV=1E0;SPV=3.7671E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:50:2:0:0%:2,0,38,6","1/1:.:387:21:344:94.25%:20,1,300,44"
2,chr1,1035169,.,TG,T,.,PASS,DP=116;SOMATIC;SS=2;SSC=44;GPV=1E0;SPV=3.8026E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:29:12:0:0%:12,0,13,0","0/1:.:87:23:39:62.9%:23,0,39,0"
3,chr1,1043223,.,CCT,C,.,PASS,DP=138;SOMATIC;SS=2;SSC=38;GPV=1E0;SPV=1.4286E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:45:4:0:0%:4,0,41,0","1/1:.:93:8:85:91.4%:8,0,84,1"
4,chr1,1223154,.,G,GAC,.,PASS,DP=79;SOMATIC;SS=2;SSC=30;GPV=1E0;SPV=8.0797E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:23:11:0:0%:11,0,12,0","0/1:.:56:25:28:52.83%:25,0,27,1"
...,...,...,...,...,...,...,...,...,...,...,...
5958,chrX,156007685,.,AC,A,.,PASS,DP=73;SOMATIC;SS=2;SSC=14;GPV=1E0;SPV=3.2025E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:20:4:0:0%:3,1,6,10","0/1:.:53:21:32:60.38%:6,15,8,24"
5959,chrY,11315316,.,C,CCT,.,PASS,DP=57;SOMATIC;SS=2;SSC=23;GPV=1E0;SPV=4.5111E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:24:11:0:0%:0,11,0,12","0/1:.:33:18:15:45.45%:0,18,0,15"
5960,chrY,11340787,.,AAC,A,.,PASS,DP=169;SOMATIC;SS=2;SSC=46;GPV=1E0;SPV=2.4816E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:56:35:0:0%:0,35,3,17","0/1:.:113:79:34:30.09%:3,76,9,25"
5961,chrY,11340868,.,TTAAAA,T,.,PASS,DP=83;SOMATIC;SS=2;SSC=32;GPV=1E0;SPV=5.8722E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:31:22:0:0%:1,21,0,9","0/1:.:52:34:18:34.62%:3,31,0,18"


In [138]:
varscan_vcf = sorting(pd.concat([varscan_snp_vcf,varscan_indel_vcf]))
varscan_vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,189392,.,ACC,A,.,PASS,DP=206;SOMATIC;SS=2;SSC=79;GPV=1E0;SPV=1.081E-8,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:78:51:0:0%:51,0,27,0","0/1:.:128:82:46:35.94%:82,0,46,0"
1,chr1,1013466,.,T,TA,.,PASS,DP=437;SOMATIC;SS=2;SSC=24;GPV=1E0;SPV=3.7671E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:50:2:0:0%:2,0,38,6","1/1:.:387:21:344:94.25%:20,1,300,44"
2,chr1,1035169,.,TG,T,.,PASS,DP=116;SOMATIC;SS=2;SSC=44;GPV=1E0;SPV=3.8026E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:29:12:0:0%:12,0,13,0","0/1:.:87:23:39:62.9%:23,0,39,0"
3,chr1,1043223,.,CCT,C,.,PASS,DP=138;SOMATIC;SS=2;SSC=38;GPV=1E0;SPV=1.4286E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:45:4:0:0%:4,0,41,0","1/1:.:93:8:85:91.4%:8,0,84,1"
4,chr1,1197765,.,C,G,.,PASS,DP=90;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.4307E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:20:20:0:0%:11,9,0,0","0/1:.:70:53:17:24.29%:23,30,5,12"
...,...,...,...,...,...,...,...,...,...,...,...
7065,chrY,11315176,.,C,T,.,PASS,DP=172;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.0766E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:74:71:2:2.74%:54,17,1,1","0/1:.:98:84:14:14.29%:72,12,12,2"
7066,chrY,11315316,.,C,CCT,.,PASS,DP=57;SOMATIC;SS=2;SSC=23;GPV=1E0;SPV=4.5111E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:24:11:0:0%:0,11,0,12","0/1:.:33:18:15:45.45%:0,18,0,15"
7067,chrY,11340787,.,AAC,A,.,PASS,DP=169;SOMATIC;SS=2;SSC=46;GPV=1E0;SPV=2.4816E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:56:35:0:0%:0,35,3,17","0/1:.:113:79:34:30.09%:3,76,9,25"
7068,chrY,11340868,.,TTAAAA,T,.,PASS,DP=83;SOMATIC;SS=2;SSC=32;GPV=1E0;SPV=5.8722E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:31:22:0:0%:1,21,0,9","0/1:.:52:34:18:34.62%:3,31,0,18"


In [143]:
mutect_vcf = read_mutect('test.filtered.vcf')
varscan_vcf = read_varscan('test.somatic.indel.Somatic.hc.vcf', 'test.somatic.snp.Somatic.hc.vcf')
vcf_somatic = merge_somatic(mutect_vcf, varscan_vcf)
vcf_somatic

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
4,chr1,54251857,.,G,A,.,PASS,"CONTQ=93;DP=255;ECNT=1;GERMQ=241;MBQ=36,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:80,0:0.012:80:32,0:48,0","0/1:89,66:0.427:155:47,31:42,35"
...,...,...,...,...,...,...,...,...,...,...,...
119,chr22,49920195,.,G,A,.,PASS,"CONTQ=93;DP=181;ECNT=1;GERMQ=169;MBQ=38,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:51,0:0.019:51:23,0:28,0","0/1:86,41:0.325:127:41,18:45,23"
120,chrX,53534643,.,G,T,.,PASS,"CONTQ=93;DP=277;ECNT=1;GERMQ=340;MBQ=35,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:96,0:0.010:96:48,0:48,0","0/1:116,56:0.327:172:58,23:58,33"
121,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
122,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"


### Test on germline

In [150]:
header_dict, vcf = read_vcf('test.germline.vcf')
info_dict = get_number(header_dict["INFO"])
format_dict = get_number(header_dict["FORMAT"])

haplotypecaller_vcf = read_haplotypecaller('test.germline.vcf')
haplotypecaller_vcf

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189
0,chr1,69511,rs75062661,A,G,1570.03,.,AC=2;AF=1.00;AN=2;DB;DP=50;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,50:50:99:1584,150,0"
1,chr1,930248,rs41285790,G,A,806.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.153e+00;DB;...,GT:AD:DP:GQ:PL,"0/1:37,33:70:99:814,0,1106"
2,chr1,942451,rs6672356,T,C,1005.03,.,AC=2;AF=1.00;AN=2;DB;DP=30;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,30:30:90:1019,90,0"
3,chr1,946247,rs2272757,G,A,735.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.893e+00;DB;...,GT:AD:DP:GQ:PL,"0/1:19,25:44:99:743,0,606"
4,chr1,952421,rs3828047,A,G,1938.03,.,AC=2;AF=1.00;AN=2;DB;DP=48;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,48:48:99:1952,144,0"
...,...,...,...,...,...,...,...,...,...,...
28202,chrX,154786335,rs150129127,C,T,1190.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=6.12;DB;DP=108...,GT:AD:DP:GQ:PL,"0/1:66,42:108:99:1198,0,1723"
28203,chrX,154791839,rs1126762,C,A,719.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=1.99;DB;DP=45;...,GT:AD:DP:GQ:PL,"0/1:21,24:45:99:727,0,574"
28204,chrX,155774618,.,A,G,2102.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=4.00;DP=140;Ex...,GT:AD:DP:GQ:PL,"0/1:69,71:140:99:2110,0,1896"
28205,chrX,156010159,.,A,G,1210.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=5.38;DP=87;Exc...,GT:AD:DP:GQ:PL,"0/1:45,32:77:99:1218,0,2871"


In [151]:
varscan_germline = read_varscan_g('test.somatic.snp.Germline.hc.vcf', 'test.somatic.indel.Germline.hc.vcf')
varscan_germline

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,13649,.,G,C,.,PASS,DP=182;SS=1;SSC=1;GPV=8.1714E-14;SPV=6.3294E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:62:48:14:22.58%:3,45,0,14","0/1:.:120:94:26:21.67%:7,87,2,24"
1,chr1,16495,.,G,C,.,PASS,DP=87;SS=1;SSC=4;GPV=1.5786E-5;SPV=3.3808E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:42:36:6:14.29%:34,2,6,0","0/1:.:45:36:9:20%:36,0,9,0"
2,chr1,16949,.,A,C,.,PASS,DP=227;SS=1;SSC=0;GPV=3.1384E-8;SPV=9.8508E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:129:111:18:13.95%:4,107,1,17","0/1:.:98:92:6:6.12%:2,90,0,6"
3,chr1,17020,.,G,A,.,PASS,DP=113;SS=1;SSC=0;GPV=4.2743E-11;SPV=9.3941E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:77:53:24:31.17%:10,43,3,21","0/1:.:36:29:7:19.44%:1,28,0,7"
4,chr1,17538,.,C,A,.,PASS,DP=176;SS=1;SSC=3;GPV=5.9742E-15;SPV=4.4453E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:82:62:19:23.46%:46,16,14,5","0/1:.:94:70:24:25.53%:61,9,19,5"
...,...,...,...,...,...,...,...,...,...,...,...
57455,chrY,56855457,.,G,A,.,PASS,DP=203;SS=1;SSC=4;GPV=5.3816E-13;SPV=3.649E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:61:51:10:16.39%:8,43,2,8","0/1:.:142:114:28:19.72%:22,92,10,18"
57456,chrY,56855461,.,C,T,.,PASS,DP=213;SS=1;SSC=0;GPV=8.3326E-23;SPV=8.0163E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:62:41:21:33.87%:6,35,1,20","0/1:.:151:107:44:29.14%:25,82,6,38"
57457,chrY,56855473,.,A,C,.,PASS,DP=189;SS=1;SSC=0;GPV=3.9605E-113;SPV=1E0,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:53:0:53:100%:0,0,5,48","1/1:.:136:0:136:100%:0,0,26,110"
57458,chrY,56855500,.,G,A,.,PASS,DP=200;SS=1;SSC=0;GPV=3.4544E-55;SPV=8.6891E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:56:16:40:71.43%:0,16,3,37","0/1:.:144:51:92:64.34%:3,48,14,78"


In [144]:
vcf_germline = merge_germline(haplotypecaller_vcf, varscan_germline)

In [145]:
vcf_germline

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189
0,chr1,69511,rs75062661,A,G,1570.03,.,AC=2;AF=1.00;AN=2;DB;DP=50;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,50:50:99:1584,150,0"
1,chr1,930248,rs41285790,G,A,806.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.153e+00;DB;...,GT:AD:DP:GQ:PL,"0/1:37,33:70:99:814,0,1106"
2,chr1,942451,rs6672356,T,C,1005.03,.,AC=2;AF=1.00;AN=2;DB;DP=30;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,30:30:90:1019,90,0"
3,chr1,946247,rs2272757,G,A,735.60,.,AC=1;AF=0.500;AN=2;BaseQRankSum=-2.893e+00;DB;...,GT:AD:DP:GQ:PL,"0/1:19,25:44:99:743,0,606"
4,chr1,952421,rs3828047,A,G,1938.03,.,AC=2;AF=1.00;AN=2;DB;DP=48;ExcessHet=3.0103;FS...,GT:AD:DP:GQ:PL,"1/1:0,48:48:99:1952,144,0"
...,...,...,...,...,...,...,...,...,...,...
20748,chrX,154465991,.,G,C,2786.03,.,AC=2;AF=1.00;AN=2;DP=72;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,70:70:99:2800,211,0"
20749,chrX,154467565,.,T,G,1723.03,.,AC=2;AF=1.00;AN=2;DP=53;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,52:52:99:1737,156,0"
20750,chrX,154467904,.,C,G,3290.03,.,AC=2;AF=1.00;AN=2;DP=100;ExcessHet=3.0103;FS=0...,GT:AD:DP:GQ:PL,"1/1:0,100:100:99:3304,300,0"
20751,chrX,154469004,.,T,C,1303.03,.,AC=2;AF=1.00;AN=2;DP=39;ExcessHet=3.0103;FS=0....,GT:AD:DP:GQ:PL,"1/1:0,39:39:99:1317,117,0"


### Test Module

In [2]:
from pyVCF import VCF_Series, VCF_DataFrame

In [16]:
varscan_snp_s = VCF_DataFrame.read_vcf('test.somatic.snp.Somatic.hc.vcf')
varscan_indel_s = VCF_DataFrame.read_vcf('test.somatic.indel.Somatic.hc.vcf')

varscan_snp_g = VCF_DataFrame.read_vcf('test.somatic.snp.Germline.hc.vcf')
varscan_indel_g = VCF_DataFrame.read_vcf('test.somatic.indel.Germline.hc.vcf')

mutect_test = VCF_DataFrame.read_vcf('test.filtered.vcf')
haplotype_test = VCF_DataFrame.read_vcf('test.germline.vcf')


In [17]:
mutect_test.set_format()
mutect_test.set_info()
mutect_test = mutect_test.unstack()
mutect_test = mutect_test.filtering()

In [18]:
mutect_test

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,35850153,.,G,T,.,PASS,"CONTQ=41;DP=159;ECNT=1;GERMQ=273;MBQ=38,30;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:61,1:0.030:62:39,0:22,1","0/1:80,5:0.067:85:43,0:37,5"
4,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
...,...,...,...,...,...,...,...,...,...,...,...
154,chrX,76428580,.,CTGAGGGCCTGAGCACCTCCGTGCAGGCCACTCCTGA,C,.,PASS,"CONTQ=93;DP=480;ECNT=1;GERMQ=514;MBQ=35,36;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:174,0:5.561e-03:174:74,0:100,0","0/1:219,49:0.188:268:127,27:92,22"
155,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
156,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"
157,chrX,85003924,.,C,T,.,PASS,"CONTQ=93;DP=483;ECNT=1;GERMQ=753;MBQ=37,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:113,0:8.669e-03:113:58,0:55,0","0/1:185,177:0.489:362:74,75:111,102"


In [19]:
haplotype_test.set_format()
haplotype_test.set_info()
haplotype_test = haplotype_test.unstack()
display(haplotype_test.shape)
haplotype_test = haplotype_test.iloc[ (row.filter_VAF(germline=True) for _, row in haplotype_test.iterrows())]

(33957, 10)

In [20]:
display(haplotype_test.shape)

(26930, 10)

In [24]:
varscan_snp_s.set_format()
varscan_snp_s.set_info()
varscan_indel_s.set_format()
varscan_indel_s.set_info()
varscan_snp_s = varscan_snp_s.filtering()
varscan_indel_s = varscan_indel_s.filtering()

In [29]:
varscan_s_test = VCF_DataFrame.concat([varscan_snp_s, varscan_indel_s]).sorting().reset_index(drop=True)
varscan_s_test

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,189392,.,ACC,A,.,PASS,DP=206;SOMATIC;SS=2;SSC=79;GPV=1E0;SPV=1.081E-8,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:78:51:0:0%:51,0,27,0","0/1:.:128:82:46:35.94%:82,0,46,0"
1,chr1,1013466,.,T,TA,.,PASS,DP=437;SOMATIC;SS=2;SSC=24;GPV=1E0;SPV=3.7671E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:50:2:0:0%:2,0,38,6","1/1:.:387:21:344:94.25%:20,1,300,44"
2,chr1,1035169,.,TG,T,.,PASS,DP=116;SOMATIC;SS=2;SSC=44;GPV=1E0;SPV=3.8026E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:29:12:0:0%:12,0,13,0","0/1:.:87:23:39:62.9%:23,0,39,0"
3,chr1,1043223,.,CCT,C,.,PASS,DP=138;SOMATIC;SS=2;SSC=38;GPV=1E0;SPV=1.4286E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:45:4:0:0%:4,0,41,0","1/1:.:93:8:85:91.4%:8,0,84,1"
4,chr1,1197765,.,C,G,.,PASS,DP=90;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.4307E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:20:20:0:0%:11,9,0,0","0/1:.:70:53:17:24.29%:23,30,5,12"
...,...,...,...,...,...,...,...,...,...,...,...
7009,chrY,11315176,.,C,T,.,PASS,DP=172;SOMATIC;SS=2;SSC=20;GPV=1E0;SPV=8.0766E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/0:.:74:71:2:2.74%:54,17,1,1","0/1:.:98:84:14:14.29%:72,12,12,2"
7010,chrY,11315316,.,C,CCT,.,PASS,DP=57;SOMATIC;SS=2;SSC=23;GPV=1E0;SPV=4.5111E-3,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:24:11:0:0%:0,11,0,12","0/1:.:33:18:15:45.45%:0,18,0,15"
7011,chrY,11340787,.,AAC,A,.,PASS,DP=169;SOMATIC;SS=2;SSC=46;GPV=1E0;SPV=2.4816E-5,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:56:35:0:0%:0,35,3,17","0/1:.:113:79:34:30.09%:3,76,9,25"
7012,chrY,11340868,.,TTAAAA,T,.,PASS,DP=83;SOMATIC;SS=2;SSC=32;GPV=1E0;SPV=5.8722E-4,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:31:22:0:0%:1,21,0,9","0/1:.:52:34:18:34.62%:3,31,0,18"


In [31]:
varscan_snp_g.set_format()
varscan_snp_g.set_info()
varscan_indel_g.set_format()
varscan_indel_g.set_info()
varscan_snp_g = varscan_snp_g.iloc[ (row.filter_VAF(germline=True) for _, row in varscan_snp_g.iterrows())]
varscan_indel_g = varscan_indel_g.iloc[ (row.filter_VAF(germline=True) for _, row in varscan_indel_g.iterrows())]

In [33]:
varscan_g_test = VCF_DataFrame.concat([varscan_snp_g, varscan_indel_g]).sorting().reset_index(drop=True)
varscan_g_test

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,NORMAL,TUMOR
0,chr1,13649,.,G,C,.,PASS,DP=182;SS=1;SSC=1;GPV=8.1714E-14;SPV=6.3294E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:62:48:14:22.58%:3,45,0,14","0/1:.:120:94:26:21.67%:7,87,2,24"
1,chr1,14907,.,A,G,.,str10,DP=47;SS=1;SSC=10;GPV=8.0864E-6;SPV=8.3752E-2,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:33:25:8:24.24%:8,17,8,0","0/1:.:14:7:7:50%:0,7,7,0"
2,chr1,17020,.,G,A,.,PASS,DP=113;SS=1;SSC=0;GPV=4.2743E-11;SPV=9.3941E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:77:53:24:31.17%:10,43,3,21","0/1:.:36:29:7:19.44%:1,28,0,7"
3,chr1,17538,.,C,A,.,PASS,DP=176;SS=1;SSC=3;GPV=5.9742E-15;SPV=4.4453E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:82:62:19:23.46%:46,16,14,5","0/1:.:94:70:24:25.53%:61,9,19,5"
4,chr1,69511,.,A,G,.,PASS,DP=114;SS=1;SSC=8;GPV=2.9295E-64;SPV=1.537E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:69:0:69:100%:0,0,46,23","1/1:.:45:2:43:95.56%:0,2,24,19"
...,...,...,...,...,...,...,...,...,...,...,...
53829,chrY,56855325,.,G,A,.,PASS,DP=473;SS=1;SSC=0;GPV=1.4916E-30;SPV=8.921E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:137:106:31:22.63%:55,51,19,12","0/1:.:336:275:61:18.15%:127,148,30,31"
53830,chrY,56855461,.,C,T,.,PASS,DP=213;SS=1;SSC=0;GPV=8.3326E-23;SPV=8.0163E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:62:41:21:33.87%:6,35,1,20","0/1:.:151:107:44:29.14%:25,82,6,38"
53831,chrY,56855473,.,A,C,.,PASS,DP=189;SS=1;SSC=0;GPV=3.9605E-113;SPV=1E0,GT:GQ:DP:RD:AD:FREQ:DP4,"1/1:.:53:0:53:100%:0,0,5,48","1/1:.:136:0:136:100%:0,0,26,110"
53832,chrY,56855500,.,G,A,.,PASS,DP=200;SS=1;SSC=0;GPV=3.4544E-55;SPV=8.6891E-1,GT:GQ:DP:RD:AD:FREQ:DP4,"0/1:.:56:16:40:71.43%:0,16,3,37","0/1:.:144:51:92:64.34%:3,48,14,78"


In [37]:
compare_columns = ['#CHROM', 'POS', 'REF', 'ALT']
other_l = varscan_g_test[compare_columns]
merge_vcf = pd.merge(haplotype_test, other_l, on=compare_columns, how='inner')
germline_2_caller = VCF_DataFrame(merge_vcf, header=haplotype_test.header, source= haplotype_test.source)


In [54]:
compare_columns = ['#CHROM', 'POS', 'REF', 'ALT']
other_l = varscan_s_test[compare_columns]
merge_vcf = pd.merge(mutect_test, other_l, on=compare_columns, how='inner')
somatic_2_caller = VCF_DataFrame(merge_vcf, header=mutect_test.header, source= mutect_test.source)

In [40]:
somatic_2_caller.header

{'fileformat': ['##fileformat=VCFv4.2'],
 'FILTER': ['##FILTER=<ID=artifact_in_normal,Description="artifact_in_normal">',
  '##FILTER=<ID=bad_haplotype,Description="Variant near filtered variant on same haplotype.">',
  '##FILTER=<ID=base_quality,Description="alt median base quality">',
  '##FILTER=<ID=chimeric_original_alignment,Description="NuMT variant with too many ALT reads originally from autosome">',
  '##FILTER=<ID=clustered_events,Description="Clustered events observed in the tumor">',
  '##FILTER=<ID=contamination,Description="contamination">',
  '##FILTER=<ID=duplicate_evidence,Description="evidence for alt allele is overrepresented by apparent duplicates">',
  '##FILTER=<ID=fragment_length,Description="abs(ref - alt) median fragment length">',
  '##FILTER=<ID=germline_risk,Description="Evidence indicates this site is germline, not somatic">',
  '##FILTER=<ID=low_avg_alt_quality,Description="Low average alt quality">',
  '##FILTER=<ID=mapping_quality,Description="ref - alt m

In [47]:
file_path = 'test.somatic.2_caller.vcf'

with open(file_path,'w+') as f:
    for k,v in somatic_2_caller.header.items():
        f.writelines('\n'.join(v))
        f.write('\n')
    # f.write('\t'.join(somatic_2_caller.columns)+'\n')

    somatic_2_caller.to_csv(f, header=True, sep='\t', index=False)
    f.close()


# somatic_2_caller

In [49]:
file_path = 'test.germline.2_caller.vcf'

with open(file_path,'w+') as f:
    for k,v in germline_2_caller.header.items():
        f.writelines('\n'.join(v))
        f.write('\n')
    # f.write('\t'.join(somatic_2_caller.columns)+'\n')

    germline_2_caller.to_csv(f, header=True, sep='\t', index=False)
    f.close()


# somatic_2_caller

In [55]:
file_path = 'hehe.vcf'
hehe = somatic_2_caller.copy()
# hehe.header = None
with open(file_path,'w+') as f:
    f.write('\n'.join(['\n'.join(lines) for _, lines in hehe.header.items()])) if hehe.header is not None else ...
    # # f.write('\t'.join(somatic_2_caller.columns)+'\n')

    # somatic_2_caller.to_csv(f, header=True, sep='\t', index=False)
    # f.close()

In [70]:
bcf =VCF_DataFrame.read_vcf('test.somatic.bcf.vcf')

def empty_format(format_list: list[str]):
    empty = True
    for i in format_list:
        if i not in ('.', './.', ' .'):
            empty =False
    return empty


bcf_f1 = bcf[~bcf.SRR22816189.str.split(':').map(empty_format)].reset_index(drop=True)
bcf_f2 = bcf_f1[~bcf_f1["2:SRR22816189"].str.split(':').map(empty_format)].reset_index(drop=True)


In [71]:
bcf_f2

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205,2:SRR22816189,2:SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.222,0.162,0.232;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:19,0:0.046:19:10,0:9,0:.:.:.:.","0/1:53,16:0.239:69:27,9:26,7:.:.:.:.","0/0:0:.:20:.:.:.:20:0%:11,9,0,0","0/1:17:.:70:.:.:.:53:24.29%:23,30,5,12"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.303,0.293,0.327;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:24,0:0.038:24:11,0:13,0:.:.:.:.","0/1:37,18:0.33:55:19,6:18,12:.:.:.:.","0/0:0:.:24:.:.:.:24:0%:14,10,0,0","0/1:18:.:56:.:.:.:38:32.14%:28,10,9,9"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.313,0.354,0.365;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:9,0:0.088:9:8,0:1,0:.:.:.:.","0/1:33,19:0.37:52:14,9:19,10:.:.:.:.","0/0:0:.:9:.:.:.:9:0%:5,4,0,0","0/1:19:.:52:.:.:.:33:36.54%:22,11,14,5"
3,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.313,0.313,0.338;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:31,0:0.03:31:18,0:13,0:.:.:.:.","0/1:43,22:0.34:65:21,11:22,11:.:.:.:.","0/0:0:.:31:.:.:.:31:0%:15,16,0,0","0/1:22:.:66:.:.:.:44:33.33%:25,19,12,10"
4,chr1,42700781,.,G,A,.,PASS,"CONTQ=86;ECNT=1;SAAF=0.02,0.04,0.037;SAPP=0.00...",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:77,0:0.013:77:37,0:40,0:.:.:.:.","0/1:183,7:0.037:190:79,3:104,4:.:.:.:.","0/0:0:.:80:.:.:.:80:0%:80,0,0,0","0/1:14:.:201:.:.:.:187:6.97%:187,0,14,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,chr22,49920195,.,G,A,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.303,0.303,0.323;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:51,0:0.019:51:23,0:28,0:.:.:.:.","0/1:86,41:0.325:127:41,18:45,23:.:.:.:.","0/0:0:.:51:.:.:.:51:0%:30,21,0,0","0/1:40:.:129:.:.:.:88:31.25%:63,25,27,13"
130,chrX,53534643,.,G,T,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.313,0.303,0.326;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:96,0:0.01:96:48,0:48,0:.:.:.:.","0/1:116,56:0.327:172:58,23:58,33:.:.:.:.","0/0:0:.:96:.:.:.:96:0%:52,44,0,0","0/1:56:.:172:.:.:.:116:32.56%:65,51,29,27"
131,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.202,0.222,0.233;SAPP=0....",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:147,0:0.006561:147:61,0:86,0:.:.:.:.","0/1:181,55:0.237:236:92,27:89,28:.:.:.:.","0/0:0:.:132:.:.:.:131:0%:67,64,0,0","0/1:34:.:217:.:.:.:183:15.67%:91,92,22,12"
132,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;ECNT=1;SAAF=0.222,0.202,0.23;SAPP=0.0...",GT:AD:AF:DP:F1R2:F2R1:GQ:RD:FREQ:DP4,"0/0:121,0:0.007872:121:45,0:76,0:.:.:.:.","0/1:137,41:0.234:178:72,17:65,24:.:.:.:.","0/0:0:.:117:.:.:.:117:0%:49,68,0,0","0/1:41:.:177:.:.:.:136:23.16%:70,66,18,23"


In [72]:
somatic_2_caller

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
4,chr1,54251857,.,G,A,.,PASS,"CONTQ=93;DP=255;ECNT=1;GERMQ=241;MBQ=36,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:80,0:0.012:80:32,0:48,0","0/1:89,66:0.427:155:47,31:42,35"
...,...,...,...,...,...,...,...,...,...,...,...
119,chr22,49920195,.,G,A,.,PASS,"CONTQ=93;DP=181;ECNT=1;GERMQ=169;MBQ=38,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:51,0:0.019:51:23,0:28,0","0/1:86,41:0.325:127:41,18:45,23"
120,chrX,53534643,.,G,T,.,PASS,"CONTQ=93;DP=277;ECNT=1;GERMQ=340;MBQ=35,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:96,0:0.010:96:48,0:48,0","0/1:116,56:0.327:172:58,23:58,33"
121,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
122,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"


In [73]:
compare_columns = ['#CHROM', 'POS', 'REF', 'ALT']
other_l = bcf_f2[compare_columns]
merge_vcf = pd.merge(somatic_2_caller, other_l, on=compare_columns, how='inner')
intersect = VCF_DataFrame(merge_vcf, header=mutect_test.header, source= mutect_test.source)
intersect

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
4,chr1,54251857,.,G,A,.,PASS,"CONTQ=93;DP=255;ECNT=1;GERMQ=241;MBQ=36,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:80,0:0.012:80:32,0:48,0","0/1:89,66:0.427:155:47,31:42,35"
...,...,...,...,...,...,...,...,...,...,...,...
119,chr22,49920195,.,G,A,.,PASS,"CONTQ=93;DP=181;ECNT=1;GERMQ=169;MBQ=38,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:51,0:0.019:51:23,0:28,0","0/1:86,41:0.325:127:41,18:45,23"
120,chrX,53534643,.,G,T,.,PASS,"CONTQ=93;DP=277;ECNT=1;GERMQ=340;MBQ=35,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:96,0:0.010:96:48,0:48,0","0/1:116,56:0.327:172:58,23:58,33"
121,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
122,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"


In [82]:
bcf_f3 = bcf_f2.iloc[:,:-2]
bcf_f3.filter_VAF()

ValueError: invalid literal for int() with base 10: '.'

In [75]:
compare_columns = ['#CHROM', 'POS', 'REF', 'ALT']
other_l = bcf_f2[compare_columns]
merge_vcf = pd.merge(somatic_2_caller, other_l, on=compare_columns, how='right')
diff = VCF_DataFrame(merge_vcf, header=mutect_test.header, source= mutect_test.source)
diff

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SRR22816189,SRR22816205
0,chr1,1197765,.,C,G,.,PASS,"CONTQ=93;DP=94;ECNT=1;GERMQ=89;MBQ=37,36;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:19,0:0.046:19:10,0:9,0","0/1:53,16:0.239:69:27,9:26,7"
1,chr1,1966436,.,C,T,.,PASS,"CONTQ=93;DP=81;ECNT=1;GERMQ=80;MBQ=36,35;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:24,0:0.038:24:11,0:13,0","0/1:37,18:0.330:55:19,6:18,12"
2,chr1,27366258,.,C,A,.,PASS,"CONTQ=93;DP=65;ECNT=1;GERMQ=30;MBQ=35,33;MFRL=...",GT:AD:AF:DP:F1R2:F2R1,"0/0:9,0:0.088:9:8,0:1,0","0/1:33,19:0.370:52:14,9:19,10"
3,chr1,37761873,.,C,T,.,PASS,"CONTQ=93;DP=104;ECNT=1;GERMQ=81;MBQ=31,33;MFRL...",GT:AD:AF:DP:F1R2:F2R1,"0/0:31,0:0.030:31:18,0:13,0","0/1:43,22:0.340:65:21,11:22,11"
4,chr1,42700781,NaN,G,A,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
129,chr22,49920195,.,G,A,.,PASS,"CONTQ=93;DP=181;ECNT=1;GERMQ=169;MBQ=38,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:51,0:0.019:51:23,0:28,0","0/1:86,41:0.325:127:41,18:45,23"
130,chrX,53534643,.,G,T,.,PASS,"CONTQ=93;DP=277;ECNT=1;GERMQ=340;MBQ=35,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:96,0:0.010:96:48,0:48,0","0/1:116,56:0.327:172:58,23:58,33"
131,chrX,76428915,.,AGCACCTCCGT,A,.,PASS,"CONTQ=93;DP=410;ECNT=1;GERMQ=452;MBQ=33,33;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:147,0:6.561e-03:147:61,0:86,0","0/1:181,55:0.237:236:92,27:89,28"
132,chrX,76429023,.,A,G,.,PASS,"CONTQ=93;DP=317;ECNT=1;GERMQ=371;MBQ=33,34;MFR...",GT:AD:AF:DP:F1R2:F2R1,"0/0:121,0:7.872e-03:121:45,0:76,0","0/1:137,41:0.234:178:72,17:65,24"
